In [1]:
'''
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
'''

'\nimport os\nos.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152\nos.environ["CUDA_VISIBLE_DEVICES"] = ""\n'

In [2]:
import gym
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
#from keras.constraints import MaxNorm
import numpy as np
import tensorflow as tf
import gc
import pickle
from keras.models import load_model

Using TensorFlow backend.


In [3]:
#Checking use of GPUs
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
sess= K.get_session()

['/gpu:0']


In [4]:
#Initializing environment
env = gym.make('Humanoid-v2')
So = env.reset()
A = env.action_space.sample()

In [5]:
#Creating the model for policy
global policy_model
policy_model = Sequential()
policy_model.add(Dense(100,input_shape = np.append(So, A).shape,kernel_initializer='random_uniform', activation = 'relu' ))
policy_model.add(Dense(30,kernel_initializer='random_uniform', activation = 'relu'))
policy_model.add(Dense(1,kernel_initializer='random_uniform', activation='linear' ))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
policy_model.compile(loss="mse", optimizer= "sgd")
print(policy_model.summary())
policy_model = load_model('policy.h5')

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for policy improvement
global policy_eligibility_traces
policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]

#Setting learning hyperparameters 
global policy_alpha
policy_alpha = 0.1
global policy_lambda
policy_lambda =  0.9

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               39400     
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 42,461
Trainable params: 42,461
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
#Creating model for values 
global value_model
value_model = Sequential()
value_model.add(Dense(100,input_shape = So.shape, kernel_initializer='random_uniform',activation = 'relu' ))
value_model.add(Dense(30, kernel_initializer='random_uniform',activation = 'relu'))
value_model.add(Dense(1,kernel_initializer='random_uniform', activation= 'sigmoid'))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
value_model.compile(loss="mse", optimizer= "sgd")
print(value_model.summary())
value_model = load_model('value.h5')

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for value improvement
global value_eligibility_traces
value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]

#Setting learning hyperparameters 
global value_alpha
value_alpha = 0.1
global value_lambda
value_lambda = 0.9

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               37700     
_________________________________________________________________
dense_5 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 40,761
Trainable params: 40,761
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
#Defining other hyperparameters

global average_reward
average_reward = 5 #pickle.load(open('average_reward.ferch','rb'))
global etaa
etaa = 0.01

In [8]:
#Auxiliary functions for weights update

def set_weights(model, weights):  
    if(model=='value'):
        i=0
        for layer in value_model.layers:
            if 'dropout' not in layer.name:
                layer.set_weights([weights[i],weights[i+1]])
                i+=2
    elif(model=='policy'):
        i=0
        for layerz in policy_model.layers:
            if 'dropout' not in layerz.name:
                layerz.set_weights([weights[i],weights[i+1]])
                i+=2
    else:
        print('error')

def get_value_weights():
    weights =[]
    for w in value_model.layers:
        weights+= w.weights
    return weights

def get_policy_weights():
    weights =[]
    for w in policy_model.layers:
        weights+= w.weights
    return weights

In [9]:
#Reward Tracking
rewardz_received = []
delta_rewardz_received = []

In [10]:
#Policy Cheats
min_policy_value = 1
max_policy_value = 2
min_policy_list, max_policy_list = [] , []

In [11]:
#Update weights
def update_weights(previous_state, action , reward, state,  terminal, skip_count):
    
    #Compute rewards
    global average_reward,rewardz_received,delta_rewardz_received
    delta_rewards = reward - average_reward*skip_count + value_model.predict(np.array([state]))[0][0] - value_model.predict(np.array([previous_state]))[0][0]    
    rewardz_received.append(reward)
    delta_rewardz_received.append(delta_rewards)
    print(delta_rewards)
    if(terminal):
        delta_rewards = reward - average_reward*skip_count + 0 - value_model.predict(np.array([previous_state]))[0][0]
    thresh = average_reward*5
    if(delta_rewards>=thresh):
        delta_rewards =thresh
    if(delta_rewards<=-thresh):
        delta_rewards=-thresh
    average_reward = average_reward + etaa*delta_rewards
    
    #Compute value updates (eligibility traces and weights)
    global value_eligibility_traces
    value_gradients = value_model.optimizer.get_gradients(value_model.output, value_model.trainable_weights)
    value_gradients = [tf.clip_by_norm(gradient, 10) for gradient in value_gradients]
    value_eligibility_traces_op = [tf.add(tf.multiply(value_eligibility_traces[i], tf.constant(value_lambda,dtype= tf.float32)),value_gradients[i] ) for i in range(len(value_eligibility_traces))] 
    value_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(value_eligibility_traces_op,feed_dict={value_model.input:np.array([previous_state])})]       
    value_weights = get_value_weights()
    value_weights_op =  [tf.add(value_weights[i], tf.multiply(tf.constant(value_alpha*delta_rewards,dtype=tf.float32), value_eligibility_traces[i])) for i in range(len(value_weights))]
    set_weights('value', sess.run(value_weights_op))
    
    #Compute policy updates (eligibility traces and weights)
    global policy_eligibility_traces
    policy_gradients = policy_model.optimizer.get_gradients(policy_model.output, policy_model.trainable_weights)
    policy_gradients = [tf.clip_by_norm(gradient, 10) for gradient in policy_gradients]
    
    #Correction on policy predictions
    global min_policy_value,max_policy_value, min_policy_list, max_policy_list
    policy_value = policy_model.predict(np.array([np.append(previous_state, action)]))[0][0]
    if(policy_value < 0):
        min_policy_value = - policy_value
        policy_value = 1
    policy_value = policy_value + min_policy_value
    if(policy_value> max_policy_value):
        max_policy_value = policy_value + 1
    policy_value = policy_value/max_policy_value
    print("pol"+str(policy_value))
    policy_value = 1/ (policy_value)
    min_policy_list.append(min_policy_value)
    max_policy_list.append(max_policy_value)

    policy_eligibility_traces_op = [tf.add(tf.multiply(tf.constant(policy_lambda,dtype=tf.float32), policy_eligibility_traces[i]), tf.multiply(tf.constant(policy_value, dtype= tf.float32), policy_gradients[i])) for i in range(len(policy_gradients))]
    policy_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(policy_eligibility_traces_op, feed_dict={policy_model.input:np.array([np.append(previous_state,action)])})]
    policy_weights = get_policy_weights()
    policy_weights_op =  [tf.add(policy_weights[i], tf.multiply( policy_eligibility_traces[i],tf.constant(policy_alpha*delta_rewards,dtype=tf.float32))) for i in range(len(policy_weights))]
    set_weights('policy', sess.run(policy_weights_op))

In [12]:
#Creating model for remembering preferences over each state
number_of_preferences = 3
global preference_model
preference_model = Sequential()
preference_model.add(Dense(100,input_shape = So.shape, activation = 'relu' ))
preference_model.add(Dense(30, activation = 'relu'))
preference_model.add(Dense(A.shape[0]*number_of_preferences, activation= 'linear'))

adam = Adam(lr=0.3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
preference_model.compile(loss="mse", optimizer=adam)
print(preference_model.summary())
preference_model = load_model('preference.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               37700     
_________________________________________________________________
dense_8 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_9 (Dense)              (None, 51)                1581      
Total params: 42,311
Trainable params: 42,311
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Selecting an action
def select_action(state):
    
    number_of_new_actions = 3
    
    predicted_preferences = preference_model.predict(np.array([state]))[0].reshape((number_of_preferences,A.shape[0]))
    
    #FIXING MATRIX INCONSISTENCIES
    low = env.action_space.low
    high = env.action_space.high
    
    for y in range(predicted_preferences.shape[0]):
        for z in range(predicted_preferences.shape[1]):
            if(predicted_preferences[y][z] < low[z]):
                predicted_preferences[y][z] = low[z]
            if(predicted_preferences[y][z] > high[z]):
                predicted_preferences[y][z] = high[z]
                
    new_actions = env.action_space.sample().reshape(1, A.shape[0])
    
    for _ in range(number_of_new_actions-1):
        new_actions= np.concatenate((new_actions,env.action_space.sample().reshape(1, A.shape[0])), axis=0 )
    
    all_actions = np.concatenate((predicted_preferences, new_actions), axis=0)
    #pickle.dump(all_actions, open('all_actions.jf','wb'))
    all_actionz = np.concatenate((np.tile(So, (all_actions.shape[0], 1)), all_actions),axis=1)
    policy_preferences = policy_model.predict(all_actionz)
    
    global min_policy_value, max_policy_value
    actions_list = []
    for i in range(all_actions.shape[0]):
        actions_list.append((all_actions[i], policy_preferences[i][0]))
    actions_list = sorted(actions_list, key = lambda x: x[1], reverse = True)
    #print("Actions ="+ str(actions_list))
    probabilities =np.array([[(x[1]+min_policy_value)/max_policy_value for x in actions_list]])[0]
    ############
    #probabilities = sess.run(tf.contrib.layers.softmax(tf.constant([[x[1]+min_policy_value for x in actions_list]])))[0]
    ############
    #print(all_actions)
    p = np.random.rand()*probabilities.sum()
    i=0
    for j in range(probabilities.shape[0]):
        p-=probabilities[j]
        if(p<0):
            i=j
    
    best_actions = np.array([x[0] for x in actions_list[0:number_of_preferences]]).flatten()
    preference_model.fit(np.array([state]), np.array([best_actions]), verbose=0)
    return actions_list[j][0]
        
    

In [ ]:
#Main loop
S = env.reset()
#action_count=0
episode_count=0
save= True
total_reward =0 
total_reward_list=[]

skip_count = 0
skip_reward=0
while(True):
    
    #renders environment 
    #env.render()
    
    
    #Selects action according to stochastic policy
    action = select_action(S)
    #action_count+=1
    
    #Takes action 
    S1, reward, done, info = env.step(action)
    
    skip_reward += reward
    skip_count += 1
    
    #Updates weights
    #if(np.random.randint(10)%5==0):
    if(np.random.randint(9)%3==0):
        update_weights(S , action, skip_reward, S1 , done, skip_count)
        skip_reward = 0
        skip_count =0
    
    S = S1
    total_reward +=reward
    
    if(done):
        
        #print('We are now at '+str(episode_count))
        gc.collect()
        print(str(episode_count)+' --total reward ='+ str(total_reward))
        total_reward_list.append(total_reward)
        total_reward=0
        
        #Resets episode
        S = env.reset()
        #action_count=0
        episode_count+=1
        
        
        
        if(episode_count%5==0):
            
            if(save):
                print('saving models')
                print('average reward '+ str(average_reward))
                
                preference_model.save('preference.h5')
                policy_model.save('policy.h5')
                value_model.save('value.h5')
                
                pickle.dump(rewardz_received, open('rewardz_received.ferch','wb'))
                pickle.dump(delta_rewardz_received, open('delta_rewardz_received.ferch','wb'))
                pickle.dump(average_reward, open('average_reward.ferch','wb'))
                pickle.dump(total_reward_list, open('total_rewards_hist','wb'))
                #pickle.dump(min_policy_list, open('min_policy','wb'))
                #pickle.dump(max_policy_list, open('max_policy','wb'))
                
                #Restarting keras session 
                K.clear_session()
                sess = K.get_session()
                preference_model = load_model('preference.h5')
                policy_model = load_model('policy.h5')
                value_model = load_model('value.h5')

        
        #Resets eligibility traces
        global value_eligibility_traces, policy_eligibility_traces
        policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]
        value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]
    

-0.309530438825
pol0.550125621259
-0.21214065952
pol0.650387734175
-0.276458545667
pol0.685092270374
-0.235621333484
pol0.750535411888
-0.0453920332358
pol0.96718005834
0 --total reward =83.31154428
-0.636951848099
pol0.977160266545
-0.338142068073
pol0.50193647734
-0.197741654104
pol0.994599861041
-3.05742380309
pol0.994880948624
-2.03344025008
pol0.999979660197
-0.658328794022
pol0.9999858507
-0.73340094599
pol0.999999319157
0.163286825091
pol0.632967130511
0.31106825391
pol0.0767393552518
0.338874393915
pol0.395468027309
-0.49413596292
pol0.147083934
-0.783134822145
pol0.326716190628
0.0789073576517
pol0.893268989119
0.0727386543289
pol0.0938373101522
1 --total reward =214.756734908
0.0307959516597
pol0.99999950541
0.0226263458813
pol0.201526070248
-0.0334903779255
pol0.999999528056
-5.22076226804
pol0.999999794876
-0.361589846655
pol0.999999922537
-0.0704258548356
pol0.480886472019
-0.608904757727
pol0.999999944093
-0.202558087718
